In [12]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA

In [2]:
clean_df = pd.read_csv('/Users/serenaliu/Documents/GitHub/FoodReceipeNLP/data/cleaned_data.csv')
clean_df.head()

,name_0,name_1,name_2,name_3,name_4,ingre_0,ingre_1,ingre_2,ingre_3,ingre_4,...,ingre_91,ingre_92,ingre_93,ingre_94,ingre_95,ingre_96,ingre_97,ingre_98,ingre_99,Cuisine
0,0.080166,0.048612,0.077307,-0.117698,0.038966,-0.151603,0.076005,0.086456,-0.012988,0.029447,...,0.124508,0.104348,0.018878,0.255176,0.185142,0.085268,-0.039565,-0.069057,-0.084318,mexican
1,0.057411,-0.062757,0.066737,0.127286,0.141621,-0.109482,0.053349,0.059184,-0.014819,0.022162,...,0.090542,0.075584,0.013055,0.181490,0.135297,0.057755,-0.026475,-0.049713,-0.060157,mexican
2,0.004638,-0.067046,0.033966,0.014586,0.062332,-0.152388,0.076698,0.088062,-0.013372,0.030849,...,0.125465,0.104699,0.017996,0.255536,0.185853,0.088936,-0.040808,-0.070232,-0.084767,mexican
3,-0.061102,-0.071237,0.181439,-0.108818,0.163737,-0.160176,0.081161,0.091251,-0.016680,0.033847,...,0.135816,0.112063,0.019098,0.272000,0.200485,0.094812,-0.044626,-0.074634,-0.092884,mexican
4,0.003240,0.039615,-0.036836,0.175529,0.026537,-0.144929,0.074185,0.079954,-0.014618,0.033069,...,0.125845,0.106721,0.023354,0.250557,0.181160,0.084074,-0.040254,-0.060790,-0.081271,mexican


In [7]:
X = clean_df.iloc[:,:105].values
y = clean_df['Cuisine'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
clf = SVC()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.16666666666666666